## 1. Imports

In [ ]:
import random
import math
import copy

import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import os
import tensorflow as tf
# import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

import torch
from kan import KAN

from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
#%load_ext autotime

In [2]:
#from tools import KAN_es
from algos import create_XY_data, alg_keras_mlp, vector_pred_NN, alg_KAN_model, vector_pred_KAN, multi_exp, alg_keras_mlp_3_output, vector_pred_KAN_3_output

## 2. Data preparation

### 2.1. Loading data

In [3]:
'''
initial_trn_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_trn.csv")
initial_vld_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_vld.csv")
initial_tst_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_tst.csv")

initial_trn_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_trn.csv")
initial_vld_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_vld.csv")
initial_tst_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_tst.csv")

initial_trn_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_trn.csv")
initial_vld_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_vld.csv")
initial_tst_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_tst.csv")
'''
# !Some troubles with *.csv naming!

initial_trn_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_trn.csv")
initial_vld_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_tst.csv")
initial_tst_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_pro.csv")

initial_trn_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_trn.csv")
initial_vld_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_tst.csv")
initial_tst_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_pro.csv")

initial_trn_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_trn.csv")
initial_vld_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_tst.csv")
initial_tst_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_pro.csv")

In [4]:
data_col_names = initial_trn_data_udus.columns

### 2.2. Scaling data

In [5]:
mm_scaler = MinMaxScaler()
mm_scaler.fit(pd.concat([initial_trn_data_fdfs,
                         initial_trn_data_udfs, 
                         initial_trn_data_udus],
                        axis=0, sort=False, ignore_index=True))

MinMaxScaler()

In [6]:
'''
scaled_trn_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_trn_data_fdfs),columns=data_col_names) 
scaled_vld_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_vld_data_fdfs),columns=data_col_names)
scaled_tst_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_tst_data_fdfs),columns=data_col_names)

scaled_trn_data_udfs = pd.DataFrame(mm_scaler.transform(initial_trn_data_udfs),columns=data_col_names)
scaled_vld_data_udfs = pd.DataFrame(mm_scaler.transform(initial_vld_data_udfs),columns=data_col_names)
scaled_tst_data_udfs = pd.DataFrame(mm_scaler.transform(initial_tst_data_udfs),columns=data_col_names)

scaled_trn_data_udus = pd.DataFrame(mm_scaler.transform(initial_trn_data_udus),columns=data_col_names)
scaled_vld_data_udus = pd.DataFrame(mm_scaler.transform(initial_vld_data_udus),columns=data_col_names)
scaled_tst_data_udus = pd.DataFrame(mm_scaler.transform(initial_tst_data_udus),columns=data_col_names)
'''
trn = pd.DataFrame(mm_scaler.transform(initial_trn_data_fdfs),columns=data_col_names) 
vld = pd.DataFrame(mm_scaler.transform(initial_vld_data_fdfs),columns=data_col_names)
tst = pd.DataFrame(mm_scaler.transform(initial_tst_data_fdfs),columns=data_col_names)

# 3. Experiment №2
* Input data: GMT.
* Models: Multi-Layer Perceptron, Kolmogorov-Arnold Network.
* Single and multiple mode predicting.

In [9]:
NUM_ITER = 25

### 3.1.1 MLP + single mode.
* MLP with 1 hidden layer.
* Seperate model per each predicting depth.
* Number neurons in hidden layer: 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024.

In [ ]:
l_algos_names=['GMT_MLP_sgl_2', 'GMT_MLP_sgl_4', 'GMT_MLP_sgl_8',
               'GMT_MLP_sgl_16', 'GMT_MLP_sgl_32', 'GMT_MLP_sgl_64', 'GMT_MLP_sgl_128',
               'GMT_MLP_sgl_256', 'GMT_MLP_sgl_512', 'GMT_MLP_sgl_1024']

l_algos=[vector_pred_NN] * 10

Data = (trn, vld, tst)

mult_data = [Data for i in range(10)]

l_geophysical_method = ['GMT' for i in range(10)]

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(10)]

l_kwargs=[{'hidden_neurons':2, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':4, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':8, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':16, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':32, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':64, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':128, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':256, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':512, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':1024, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05}
          ]

l_metrics_names=['rmse', 'mae', 'mape', 'r2'] #['mae', 'rmse', 'r2', 'mape']

num_iter=NUM_ITER

In [ ]:
full_df_1_1 = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

In [ ]:
full_df_1_1.to_excel('full_metrics_exp2_mlp_sgl.xlsx')
#pd.read_excel('full_metrics.xlsx').drop('Unnamed: 0', axis=1)

In [ ]:
aggr_df_1_1 = full_df_1_1.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df_1_1.to_excel('aggr_metrics_exp2_mlp_sgl.xlsx')
aggr_df_1_1

### 3.1.2 MLP + multiple mode.
* MLP with 1 hidden layer.
* Only one model with multiple output.
* Number neurons in hidden layer: 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024.

In [ ]:
l_algos_names=['GMT_MLP_mul_2', 'GMT_MLP_mul_4', 'GMT_MLP_mul_8',
               'GMT_MLP_mul_16', 'GMT_MLP_mul_32', 'GMT_MLP_mul_64', 'GMT_MLP_mul_128',
               'GMT_MLP_mul_256', 'GMT_MLP_mul_512', 'GMT_MLP_mul_1024']

l_algos=[alg_keras_mlp_3_output] * 10

Data = (trn, vld, tst)

mult_data = [Data for i in range(10)]

l_geophysical_method = ['GMT' for i in range(10)]

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(10)]

l_kwargs=[{'hidden_neurons':2, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':4, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':8, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':16, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':32, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':64, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':128, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':256, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':512, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'hidden_neurons':1024, 'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05}
          ]

l_metrics_names=['rmse', 'mae', 'mape', 'r2'] #['mae', 'rmse', 'r2', 'mape']

num_iter=NUM_ITER

In [ ]:
full_df_1_2 = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

In [ ]:
full_df_1_2

In [ ]:
full_df_1_2.to_excel('full_metrics_exp2_mlp_mul.xlsx')
#pd.read_excel('full_metrics.xlsx').drop('Unnamed: 0', axis=1)

In [ ]:
aggr_df_1_2 = full_df_1_2.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df_1_2.to_excel('aggr_metrics_exp2_mlp_mul.xlsx')
aggr_df_1_2

### 3.2.1 KAN + single mode.
* KAN with 1 hidden layer.
* Seperate model per each predicting depth.
* Number neurons in hidden layer: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10.

In [10]:
l_algos_names=['GMT_KAN_sgl_1', 'GMT_KAN_sgl_2', 'GMT_KAN_sgl_3', 'GMT_KAN_sgl_4', 'GMT_KAN_sgl_5', 
               'GMT_KAN_sgl_6', 'GMT_KAN_sgl_7', 'GMT_KAN_sgl_8', 'GMT_KAN_sgl_9', 'GMT_KAN_sgl_10']

l_algos=[vector_pred_KAN, vector_pred_KAN, vector_pred_KAN, vector_pred_KAN, vector_pred_KAN,
         vector_pred_KAN, vector_pred_KAN, vector_pred_KAN, vector_pred_KAN, vector_pred_KAN]

Data = (trn, vld, tst)

mult_data = [Data for i in range(10)]

l_geophysical_method = ['GMT' for i in range(10)]

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(10)]

l_kwargs=[{'K':3, 'hidden_neurons':1, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':2, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':3, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':4, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':5, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':6, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':7, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':8, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':9, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':10, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          ]

l_metrics_names=['rmse', 'mae', 'mape', 'r2'] #['mae', 'rmse', 'r2', 'mape']

num_iter=NUM_ITER

In [ ]:
full_df_2_1 = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

In [9]:
full_df_2_1

,alg_name,iter,mae,rmse,r2
0,GMT_KAN_hn_1,1,0.14007,0.17543,0.64241
1,GMT_KAN_hn_1,2,0.13545,0.17022,0.65990
2,GMT_KAN_hn_1,3,0.13684,0.17176,0.65548
3,GMT_KAN_hn_1,4,0.13375,0.16864,0.66194
4,GMT_KAN_hn_1,5,0.13062,0.16459,0.67304
5,GMT_KAN_hn_2,1,0.13964,0.17535,0.64265
6,GMT_KAN_hn_2,2,0.14016,0.17544,0.64409
7,GMT_KAN_hn_2,3,0.13972,0.17585,0.64022
8,GMT_KAN_hn_2,4,0.13650,0.17147,0.65573
9,GMT_KAN_hn_2,5,0.13235,0.16684,0.66684


In [10]:
full_df_2_1.to_excel('full_metrics_exp2_kan_sgl.xlsx')
#pd.read_excel('full_metrics.xlsx').drop('Unnamed: 0', axis=1)

In [11]:
aggr_df_2_1 = full_df_2_1.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df_2_1.to_excel('aggr_metrics_exp2_kan_sgl.xlsx')
aggr_df_2_1

mae                rmse                  r2          
                   mean       std      mean       std      mean       std
alg_name                                                                 
GMT_KAN_hn_1   0.135346  0.003516  0.170128  0.003990  0.658554  0.011104
GMT_KAN_hn_10  0.136922  0.004401  0.172018  0.005120  0.652232  0.016499
GMT_KAN_hn_2   0.137674  0.003315  0.172990  0.003869  0.649906  0.011191
GMT_KAN_hn_5   0.137676  0.004428  0.172974  0.005050  0.649946  0.014508

### 3.2.2 KAN + multiple mode.
* KAN with 1 hidden layer.
* Only one model with multiple output.
* Number neurons in hidden layer: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10.

In [ ]:
l_algos_names=['GMT_KAN_mul_1', 'GMT_KAN_mul_2', 'GMT_KAN_mul_3', 
               'GMT_KAN_mul_4', 'GMT_KAN_mul_5', 'GMT_KAN_mul_6', 'GMT_KAN_mul_7', 
               'GMT_KAN_mul_8', 'GMT_KAN_mul_9', 'GMT_KAN_mul_10']

l_algos=[vector_pred_KAN_3_output] * 10

Data = (trn, vld, tst)

mult_data = [Data for i in range(3)]

l_geophysical_method = ['GMT' for i in range(3)]

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(3)]

l_kwargs=[{'K':3, 'hidden_neurons':1, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':2, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':3, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':4, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':5, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':6, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':7, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':8, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':9, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':10, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          ]

l_metrics_names=['rmse', 'mae', 'mape', 'r2'] #['mae', 'rmse', 'r2', 'mape']

num_iter=NUM_ITER

In [ ]:
full_df_2_2 = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

In [ ]:
full_df_2_2

In [ ]:
full_df_2_2.to_excel('full_metrics_exp2_kan_mul.xlsx')
#pd.read_excel('full_metrics.xlsx').drop('Unnamed: 0', axis=1)

In [ ]:
aggr_df_2_2 = full_df_2_2.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df_2_2.to_excel('aggr_metrics_exp2_kan_mul.xlsx')
aggr_df_2_2